In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mplt
import seaborn as sbn
from scipy import stats

pd.set_option('float_format', '{:f}'.format)

In [ ]:
#df = pd.read_csv("Parcels.csv", converters={'LANDUSE_DESC': str.strip})

import requests
from io import StringIO

orig_url='https://drive.google.com/file/d/17PjftoPhdYPevFe6lj_60WSGkYwHhh0_/view?usp=sharing'
file_id = orig_url.split('/')[-2]
dwn_url ='https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
df = pd.read_csv(csv_raw, converters={'LANDUSE_DESC': str.strip})

df.columns = [x.encode('utf-8').decode('ascii', 'ignore') for x in df.columns]

pd.set_option('display.max_columns', None)

df.info()

In [ ]:
df.head()

#### Basic info:
* 45 columns, 120,783 rows
* Useful fields:
    * OBJECTID
    * PARCEL_ID
    * SUM_ACRE
    * LAND_USE AND LANDUSE_DESC
    * LAND_VALUE, BLDG_VALUE and TOTAL_VALU
    * DATE_SOLD AND SALE_PRICE (n/a for most records)
    * SITE_ADDRE (street address only)
* Potentially useful fields (would need to know what these are and get definitions of the code values):
    * DIST_CODE
    * NBR_CODE
    * CLASS_CODE
* Wrong datatypes:
    * OBJECTID (s/b object)
    * PARCEL_ID (s/b object)
    * DATE_SOLD (s/b date)
    * DIST_CODE (s/b object)

### DATA PREP

In [ ]:
#Convert datatypes.

obj_col = ['OBJECTID', 'PARCEL_ID', 'DIST_CODE']
for i in obj_col:
    df[i] = df[i].astype('object')

df['DATE_SOLD'] = df['DATE_SOLD'].replace({0:np.nan})

df['DATE_SOLD'] = pd.to_datetime(df['DATE_SOLD'].astype(str), format='%Y%m%d')

In [ ]:
#Keep only useful fields.

df1 = df[['OBJECTID','PARCEL_ID','SUM_ACRE', 'LAND_USE', 'LANDUSE_DESC', 'LAND_VALUE', 'BLDG_VALUE', 'TOTAL_VALU',
          'DATE_SOLD', 'SALE_PRICE', 'SITE_ADDRE', 'DIST_CODE', 'NBR_CODE', 'CLASS_CODE']]

df1.info()

In [ ]:
#Which parcels have residential land uses?

df1_ordered_by_landuse_desc = df1.filter(['LANDUSE_DESC'], axis=1)
df1_ordered_by_landuse_desc_sorted = df1_ordered_by_landuse_desc.sort_values(['LANDUSE_DESC'], ascending=[1])
df1_ordered_by_landuse_desc_sorted['LANDUSE_DESC'].unique()

#### Keep only records with these land uses:

* Some 'AG/ ': '1-FAMILY', '1-MH OR MH SITE', '2-FAMILY', 'MULTIPLE DWG'S'
* Some 'COM/ ': 'APARTMENT-DWG CONV', 'APARTMENT-GARDEN', 'APARTMENT-GARDEN S42', 'APARTMENT-HIGH RISE', 'APT-HIGH RISE S42', 'DWNTWN ROW-ATTACHED', DWNTWN ROW-DETACHED', 'LIVING ACCOMM', 'LIVING ACCOMN S42'
* All 'RES/ ...' (some (e.g., RES/ GROUP LIVING) may not be appropriate, but that's a decision for someone else.
* All 'VAC 'RES/ ...' (some may not be appropriate, but that's a decision for someone else)

In [ ]:
#Create new dataframe with just residential parcels.

res_uses = ['AG/ 1-FAMILY', 'AG/ 1-MH OR MH SITE', 'AG/ 2-FAMILY', "AG/ MULTIPLE DWG'S", 'COM/ APARTMENT-DWG CONV',
           'COM/ APARTMENT-GARDEN', 'COM/ APARTMENT-GARDEN S42', 'COM/ APARTMENT-HIGH RISE', 'COM/ APT-HIGH RISE S42',
           'COM/ DWNTWN ROW-ATTACHED', 'COM/ DWNTWN ROW-DETACHED', 'COM/ LIVING ACCOMM', 'COM/ LIVING ACCOMN S42',
           'RES/ 1-FAMILY', 'RES/ 1-FAMILY S42', 'RES/ 1-MH OR MH SITE', 'RES/ 2-FAMILY', 'RES/ 2-FAMILY MULTI UNITS',
           'RES/ 2-FAMILY S42', 'RES/ 2-MH OR MH SITES', 'RES/ 3-FAMILY', 'RES/ 3-FAMILY S42', 'RES/ 4-FAMILY',
           'RES/ CONDOMINIUM', 'RES/ DCLT LSHLD IMPROV', 'RES/ DWG + 1-MBL HM', 'RES/ DWG + 2-MBL HMS', 'RES/ ESTATE',
           'RES/ GROUP LIVING', 'RES/ HISTORICAL', 'RES/ HOMEOWNERS ASSOC IMP', 'RES/ LSHLD IMPROV', 'RES/ MOBILE HOME',
           "RES/ MULTIPLE DWG'S", 'RES/ PATIO HOME W/ LAND', 'RES/ RESIDENTIAL (UNDIFF)', 'RES/ RURAL RESIDENTIAL',
           'RES/ TOWNHOUSE W/ LAND', 'RES/ TOWNHOUSE W/ LND S42', 'VAC RES/ < 10 ACRES', 'VAC RES/ DEVELOPER',
           'VAC RES/ HOMEOWNERS ASSOC', 'VAC RES/ LOT-SML TR/REAR', 'VAC RES/ LOT-SML TR/SIDE', 'VAC RES/ LOT-SML TRACT',
           'VAC RES/ RURAL < 10 ACRES', 'VAC RES/ UNDERWATER LANDS', 'VAC RES/ W/ SML IMPROV']

df_res = df1[df1['LANDUSE_DESC'].isin(res_uses)]

df_res.info()

#### Basic info:
* 14 columns (dropped 31)
* 108,203 rows (dropped only 12,580 - this is surprising)

#### Summary statistics:

In [ ]:
df_res.describe()

#### Missing (null) values

In [ ]:
# Summary of null data, by count and percentage:

nulltotal = df_res.isnull().sum().sort_values(ascending=False)
nullpercent = (df_res.isnull().sum()*100/df_res.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([nulltotal, nullpercent], axis=1, keys=['Total', 'Percent'])

missing_data.head()

In [ ]:
#df_res.head()

df_res_ordered_by_date_sold = df_res.filter(['DATE_SOLD'], axis=1)
df_res_ordered_by_date_sold_sorted = df_res_ordered_by_date_sold.sort_values(['DATE_SOLD'], ascending=[1])
df_res_ordered_by_date_sold_sorted['DATE_SOLD'].unique()

* SUM_ACRE has <10 null's - should delete these records.

* DATE_SOLD has many null's (and earliest values are 1997, which is probably ok for our needs) - don't delete these records.

In [ ]:
df_res.dropna(subset=['SUM_ACRE'], inplace=True)

#### Any duplicate rows?

In [ ]:
df_res.duplicated().sum()

#### Are there outliers?

In [ ]:
#df_res.boxplot(column=['SUM_ACRE'])
#mplt.show

df_test = pd.DataFrame(np.random.rand(10, 5), columns=["SUM_ACRE", "LAND_VALUE", "BLDG_VALUE", "TOTAL_VALU", "SALE_PRICE"])
df_test.plot.box()

There are outliers, but for these fields, extreme values are reasonable.

### EXPLORATION

#### Distributions of quantitative variables

In [ ]:
fig = mplt.figure(figsize = (30,20))
ax = fig.gca()
df_res.hist(ax=ax)
mplt.show()

##### Observations:
All quantitative variables are heavily skewed.

#### Categorical values by count

In [ ]:
# LANDUSE_DESC is a key categorical variable. DIST_CODE, NBR_CODE and CLASS_CODE may be useful; don't know yet.

from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 12), dpi=80, facecolor='w', edgecolor='k')

df_res['LANDUSE_DESC'].value_counts().plot(kind='barh')

##### Observations:
* LANDUSE_DESC:
    * RES/ 1-FAMILY is by far most prevalent residential category.
    * Next: RES/ TOWNHOUSE W/ LAND, VAC RES/ LOT-SML TRACT

#### Categorical values by quantitative variables

In [ ]:
# LANDUSE_DESC is a key categorical variable. DIST_CODE, NBR_CODE and CLASS_CODE may be useful; don't know yet.
# TOTAL_VALU is a key quantitative variable. May also want look at SUM_ACRE.
# Will look at mean values for each category.

#figure(num=None, figsize=(8, 12), dpi=80, facecolor='w', edgecolor='k')
#x = df_res.groupby('LANDUSE_DESC').mean().reset_index()
#mplt.bar(range(len(x)), x['TOTAL_VALU'], color="blue")
#mplt.xticks(range(len(x)), x['LANDUSE_DESC'])
#mplt.ylabel('Mean')

#Can't get bar chart to be readable. Will just print results.

df_res[["LANDUSE_DESC", "TOTAL_VALU"]].groupby("LANDUSE_DESC").mean().style.format('{0:,.0f}')

##### Observations:
* Pattern is what would be expected:
    * Highest values are apartments, downtown, estate.
    * COM / LIVING ACCOMN S42 also high value - what is this?
    * Lowest values are vacant, mobile home, etc.

#### Relationships among quantitative variables

In [ ]:
# Correlogram

mplt.style.use('seaborn-colorblind')
sbn.pairplot(df_res[['SUM_ACRE','LAND_VALUE','BLDG_VALUE','TOTAL_VALU','SALE_PRICE']], kind="scatter", diag_kind = 'kde', \
             plot_kws = {'alpha': 0.33, 's': 80, 'edgecolor': 'k'}, height = 4)
mplt.show()

##### Observations:

* Not surprisingly, SALE_PRICE and the 3 value fields are all highly correlated.
* Surprisingly, SUM_ACRE seems to be weakly correlated, at best, with other quantitative fields.